<a href="https://colab.research.google.com/github/Behnaz81/MachineLearningDaily/blob/main/day11_pipeline_miniproject/MachineLearning11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project to Test Some Models

In this project I tried to use an interface to test some of the NLP models using gradio.

In [ ]:
!pip install gradio

In [ ]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [19]:
import gradio as gr
from transformers import pipeline
import os
import pandas as pd

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis")
fill_mask_pipeline = pipeline("fill-mask")
ner_pipeline = pipeline("ner", grouped_entities=True)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set

In [20]:
log_file = "logs.csv"
if not os.path.exists(log_file):
    df = pd.DataFrame(columns=["Sentence", "Model", "Output"])
    df.to_csv(log_file, index=False)

In [21]:
def analysis(Sentence, Model):
  if Model == "Sentiment Analysis":
    output = sentiment_pipeline(Sentence)[0]['label']
  elif Model == "Fill-Mask":
    output = fill_mask_pipeline(Sentence)[0]['sequence']
  elif Model == "Named Entity Recognition":
    results = ner_pipeline(Sentence)
    output = []
    for r in results:
        word = r["word"]
        label = r['entity_group']
        output.append({'word':word, 'label': label})
  else:
    if not Sentence:
      output = "Invalid input. Enter a sentence."
    elif not Model:
      output = "Invalid input. Choose a model."
    else:
      output = "Invalid input!"

  if isinstance(output, dict):
      save_output = str(output)
  else:
      save_output = output

  df = pd.read_csv(log_file)
  df_new = pd.DataFrame([{"Sentence": Sentence, "Model": Model, "Output": save_output}])
  df = pd.concat([df, df_new], ignore_index=True)
  df.to_csv(log_file, index=False)

  return output

In [22]:
demo = gr.Interface(
    fn=analysis,
    inputs=[
        "text",
        gr.Radio(["Sentiment Analysis", "Fill-Mask", "Named Entity Recognition"], label="Model"),
    ],
    outputs="text"
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15e8e6dfe659c9bfc9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
